# 1. 합성공 계층

- 기존 Affine 계층은 데이터의 형상이 무시된다.
- 합성공 계층은 형상을 유지한다. 이미지도 3차원 데이터로 입력받으며, 마찬가지로 다음 계층에도 3차원 데이터로 전달합니다. 그래서 CNN에서는 이미지처럼 형상을 가진 데이터를 제대로 이해할 가능성이 있다.
- CNN에서는 합성곱 계층의 입출력 데이터를 특징 맵(feature map)이라고 합니다. 합성곱 계층의 입력 데이터를 입력 특징 맵, 출력 데이터를 출력 특징 맵이라고 하는 식이다.

### 1. 합성곱 연산

- 합성곱 연산은 이미지 처리에서 말하는 필터 연산에 해당한다.
- Affine 신경망에서는 가중치 매개변수와 편향이 존재하는데, CNN에서는 피렅의 매개변수가 그 동안의 가중치에 해당한다. CNN에도 편향이 존재한다.

### 2. 패딩

- 패딩은 주로 출력 크기를 조정할 목적으로 사용한다. 

### 3. 스트라이드

- 필터를 적용하는 위치의 간격

입력 크기를 (H, W), 필터 크기를 (FH, FW), 출력 크기를(OH, OW), 패딩을 P, 스트라이드를 S라 하면 출력 크기는 다음 식으로 계산한다.

OH = (H + 2P - FH) / S + 1

OW = (W + 2P - FW) / S + 1

위 식의 출력 값은 정수로 떨어져야 한다.

### 4. 3차원 데이터의 합성곱 연산

- 3차원 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널 수가 같아야 한다는 것입니다. 이 예는 모두 3개로 일치합니다. 한편, 필터 자체 크기는 원하는 값으로 설정할 수 있다.

### 5. 블록으로 생각하기

### 1장의 필터

<img src="./cnn.png">

- 입력 데이터: (C, H, W)
- 필터: (C, FH, FW)
- 출력 데이터: (1, OH, OW)

### FN개의 필터 + Bias

<img src="./cnn2.png">

- 입력 데이터: (C, H, W)
- 필터: (FN, C, FH, FW)
- 중간 출력 데이터: (FN, OH, OW)
- 편향: (FN, 1, 1)
- 출력 데이터: (FN, OH, OW)

### 6. 배치 처리

- 신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리한다. 완전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고 미니배치 방식의 학습도 지원하도록 했다.

<img src="./cnn3.png">

# 2. 풀링 계층

- 풀링은 세로, 가로 방향의 공간을 줄이는 연산이다. 예를 들어 2 x 2 영역을 원소 하나로 집약하여 공간 크기를 줄입니다.

- 풀링 계층의 특징(max pooling, average pooling)

    1) 학습해야 할 매개변수가 없다. 

    2) 채널 수가 변하지 않는다.

    3) 입력의 변화에 영향을 적게 받는다.

# 3. 합성곱 / 풀링 계층 구현

In [1]:
import numpy as np

### 1. 4차원 배열

In [2]:
x = np.random.rand(10, 1, 28, 28) #무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape
x[1].shape

(1, 28, 28)

### 2. im2col로 데이터 전개하기

<img src="./cnn4.png">

<img src="cnn5.png">

im2col은 필터링하기 좋게 입력 데이터를 전개한다.

1) 구체적으로는 입력 데이터에서 필터를 적용하는 영역을 한 줄로 늘어놓습니다. 이 전개를 필터를 적용하는 모든 영역에서 수행하는 것이 im2col이다.

2) 필터를 세로로 1열로 전개하고 im2col이 전개한 데이터와 행렬 내적을 계산한다.

3) 마지막으로 출력 데이터를 변형한다.

### im2col 함수의 인터페이스

im2col(input_data, filter_h, filter_w, stride=1, pad=0)

In [4]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

In [5]:
x1 = np.random.rand(1, 3, 7, 7) # 데이터 수, 채널 수, 높이, 너비
col1 = im2col(x1, 5, 5, stride=1, pad=0)

In [6]:
print(col1.shape)

(9, 75)


In [7]:
x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


### 2. Convolution

In [8]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = apd
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T
        out = np.odt(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

## 3. Pooling

<img src="cnn6.png">

In [13]:
class Pooling: 
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)
        
        # 전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h * self.pool_w)
        
        # max
        out  = np.max(col, axis=1)
        
        # transpose
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
        
        return out

# 4. CNN 구현하기

In [14]:
from collections import OrderedDict

In [15]:
from common import layers

In [18]:
from common.layers import Relu, Affine, SoftmaxWithLoss

In [22]:
class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28),
                 conv_param={
                     'filter_num': 30,
                     'filter_size': 5,
                     'pad': 0,
                     'stride': 1
                 }, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        
        conv_output_size = (input_size - filter_size + 2 * filter_pad) / filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        # weights
        self.params = {}
        
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_num)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.parmas['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.parmas['b3'] = np.zeros(output_size)
        
        # layers
        self.layers = OrderDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.params['b1'],
                                           conv_param['stride'],
                                           conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        
        self.last_layer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.last_layer.forward(y, t)
    
    def gradient(self, x, t):
        
        # forward
        self.loss(x, t)
        
        # backword
        dout = 1
        dout = self.last_layer.backward(out)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            dout = layer.backward(dout)
            
        grads = {}
        
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        gards['b3'] = self.layers['Affine2'].db
        
        return grads